In [11]:
import numpy as np
import operator
# from numpy.linalg import norm
from numpy.linalg import inv
from numpy.linalg import det
import math
from functools import reduce
import pandas as pd
# %matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
plt.style.use('seaborn-whitegrid')

norm = lambda A: np.linalg.norm(A, ord=2)

### Метод Якоби

In [28]:
def get_H(n, precision):
    # функция строит матрицу Гильберта размером n x n и округляет значения до precision знаков после запятой
    a = lambda i, j: round(1 / (i + j + 1), precision)
    return np.array([[a(i, j) for j in range(n)] for i in range(n)])

def get_symmetric(n, rng=1, seed=None):
    # строит произвольную симметричную матрицу n x n, можно задать seed
    np.random.seed(seed=seed)
    A = np.random.rand(n, n) - 0.5 * np.ones((n, n))
    return (A + A.T) * rng

In [13]:
def rotation_matrix(n, cosp, sinp, i, j):
    # матрица вращения
    A = np.eye(n)
    A[i][i] =  cosp
    A[i][j] = -sinp
    A[j][i] =  sinp
    A[j][j] =  cosp
    return A

In [14]:
def make_zero(A, i, j):
    # обнуляет a_ij
    n = len(A)
    y = A[i][i] - A[j][j]
    x = -2 * A[i][j]
    if y == 0:
        cosp = sinp = 1 / math.sqrt(2)
    else:
        cosp = math.sqrt((1 + abs(y) / math.sqrt(x**2 + y**2)) / 2)
        sinp = np.sign(x * y) * abs(x) / 2 / cosp / math.sqrt(x**2 + y**2)
    
    Tij = rotation_matrix(n, cosp, sinp, i, j)
    
    A = Tij @ A @ inv(Tij)
    return A
    

In [15]:
def max_R(A):
    # Возвращает наибольший радиус Гершгорина и номер строки
    n = len(A)
    max_r = 0
    row = 0
    for i in range(n):
        if sum(np.abs(A[i][0:i])) + sum(np.abs(A[i][i + 1:n])) > max_r:
            max_r = sum(np.abs(A[i][0:i])) + sum(np.abs(A[i][i + 1:n]))
            row = i

    return (max_r, row)

In [16]:
def get_eig_bounds(A):
    # Возвращает отрезок, в котором лежат все с.ч.
    n = len(A)
    l = A[0][0]
    r = A[0][0]

    for i in range(n):
        R = sum(np.abs(A[i][0:i])) + sum(np.abs(A[i][i + 1:n]))
        l = min(l, A[i][i] - R)
        r = max(r, A[i][i] + R)
    
    return l, r

In [17]:
def jacobi(A, eps):
    # Циклический метод
    n = len(A)
    iters = 0

    while max_R(A)[0] > eps:
        for i in range(n):
            for j in range(i + 1, n):
                A = make_zero(A, i, j)
                iters += 1

    return sorted(np.diagonal(A)), iters

In [18]:
def jacobi2(A, eps):
    # Стратегия "преград-барьеров"
    n = len(A)
    curr_eps = np.max(np.abs(A)) / 2
    iters = 0

    while max_R(A)[0] > eps:
        for i in range(n):
            for j in range(i + 1, n):
                if abs(A[i][j]) > curr_eps:
                    A = make_zero(A, i, j)
                    iters += 1

        curr_eps /= 2

    return sorted(np.diagonal(A)), iters

In [19]:
def check_results(A, eig):
    R, ind =  max_R(A)
    m, M = get_eig_bounds(A)
    print(f"Actual:     {min(eig):.14f} {max(eig):.14f}")
    print(f"Gershgorin: {m:.14f} {M:.14f}")
    in_bounds = np.all(eig >= m) and np.all(eig <= M)
    print("in the bounds" if in_bounds else "not in the bounds")
    print("diff with np:", norm(np.array(eig) - np.array(sorted(np.linalg.eig(matrix)[0])))) 
        

***
#### Тесты

In [20]:
matrix = get_H(100, 10)
eps = 1e-10

eigenvalues, iters = jacobi(matrix, eps)

print("iterations:", iters)
res = sorted(eigenvalues)
check_results(matrix, res)

iterations: 24750
Actual:     -0.00000000052737 2.18269609773652
Gershgorin: -3.53061184110000 5.18737751760000
in the bounds
diff with np: 3.538381133583424e-12


In [21]:
matrix = get_H(100, 10)
eps = 1e-10

eigenvalues, iters = jacobi2(matrix, eps)

print("iterations:", iters)
res = sorted(eigenvalues)
check_results(matrix, res)

iterations: 11706
Actual:     -0.00000000052718 2.18269609773655
Gershgorin: -3.53061184110000 5.18737751760000
in the bounds
diff with np: 4.600012075332964e-12


In [22]:
n = 50
matrix = get_symmetric(n, 10, seed=42)
eps = 1e-7

eigenvalues, iters = jacobi(matrix, eps)

print("iterations:", iters)
res = sorted(eigenvalues)
check_results(matrix, res)

iterations: 8575
Actual:     -55.02261176726022 50.04339623514577
Gershgorin: -196.98770767887771 204.83850757673696
in the bounds
diff with np: 7.749451907356228e-13


In [23]:
n = 50
matrix = get_symmetric(n, 10, seed=42)
eps = 1e-7

eigenvalues, iters = jacobi2(matrix, eps)

print("iterations:", iters)
res = sorted(eigenvalues)
check_results(matrix, res)

iterations: 4809
Actual:     -55.02261176726005 50.04339623514575
Gershgorin: -196.98770767887771 204.83850757673696
in the bounds
diff with np: 6.671172275658177e-13


In [24]:
matrix = get_H(10, 20)

for eps in np.logspace(-2, -7, 15, endpoint=False):
    eigenvalues, iters = jacobi(matrix, eps)
    diff = norm(np.array(eigenvalues) - np.array(sorted(np.linalg.eig(matrix)[0])))
    print( f"eps: {eps:.10e}   ", "iterations:", "{:6}    ".format(iters), "diff with np:", diff)

eps: 1.0000000000e-02    iterations:     90     diff with np: 1.5165367622145836e-05
eps: 4.6415888336e-03    iterations:     90     diff with np: 1.5165367622145836e-05
eps: 2.1544346900e-03    iterations:     90     diff with np: 1.5165367622145836e-05
eps: 1.0000000000e-03    iterations:     90     diff with np: 1.5165367622145836e-05
eps: 4.6415888336e-04    iterations:    135     diff with np: 3.512722935726444e-09
eps: 2.1544346900e-04    iterations:    135     diff with np: 3.512722935726444e-09
eps: 1.0000000000e-04    iterations:    135     diff with np: 3.512722935726444e-09
eps: 4.6415888336e-05    iterations:    135     diff with np: 3.512722935726444e-09
eps: 2.1544346900e-05    iterations:    135     diff with np: 3.512722935726444e-09
eps: 1.0000000000e-05    iterations:    135     diff with np: 3.512722935726444e-09
eps: 4.6415888336e-06    iterations:    135     diff with np: 3.512722935726444e-09
eps: 2.1544346900e-06    iterations:    135     diff with np: 3.51272293

In [25]:
matrix = get_H(10, 20)

for eps in np.logspace(-2, -7, 15, endpoint=False):
    eigenvalues, iters = jacobi2(matrix, eps)
    diff = norm(np.array(eigenvalues) - np.array(sorted(np.linalg.eig(matrix)[0])))
    print( f"eps: {eps:.10e}   ", "iterations:", "{:6}    ".format(iters), "diff with np:", diff)

eps: 1.0000000000e-02    iterations:     39     diff with np: 0.0017912244666478342
eps: 4.6415888336e-03    iterations:     44     diff with np: 0.0017897082482147913
eps: 2.1544346900e-03    iterations:     52     diff with np: 0.0005539220270549594
eps: 1.0000000000e-03    iterations:     57     diff with np: 9.67653746819616e-05
eps: 4.6415888336e-04    iterations:     63     diff with np: 9.690331589305865e-05
eps: 2.1544346900e-04    iterations:     67     diff with np: 9.69358596836561e-05
eps: 1.0000000000e-04    iterations:     73     diff with np: 1.3040765110594776e-05
eps: 4.6415888336e-05    iterations:     78     diff with np: 3.7975491274791704e-06
eps: 2.1544346900e-05    iterations:     78     diff with np: 3.7975491274791704e-06
eps: 1.0000000000e-05    iterations:     82     diff with np: 3.7974939710950937e-06
eps: 4.6415888336e-06    iterations:     91     diff with np: 3.6961464518888157e-07
eps: 2.1544346900e-06    iterations:     97     diff with np: 7.630210706

In [26]:
n = 20
matrix = get_symmetric(n, 10, seed=123)

for eps in np.logspace(-2, -7, 7):
    eigenvalues, iters = jacobi(matrix, eps)
    diff = norm(np.array(eigenvalues) - np.array(sorted(np.linalg.eig(matrix)[0])))
    print( f"eps: {eps:.10e}   ", "iterations:", "{:6}    ".format(iters), "diff with np:", diff)

eps: 1.0000000000e-02    iterations:    950     diff with np: 3.190980905024334e-11
eps: 1.4677992676e-03    iterations:    950     diff with np: 3.190980905024334e-11
eps: 2.1544346900e-04    iterations:    950     diff with np: 3.190980905024334e-11
eps: 3.1622776602e-05    iterations:    950     diff with np: 3.190980905024334e-11
eps: 4.6415888336e-06    iterations:   1140     diff with np: 1.9324582278024977e-13
eps: 6.8129206906e-07    iterations:   1140     diff with np: 1.9324582278024977e-13
eps: 1.0000000000e-07    iterations:   1140     diff with np: 1.9324582278024977e-13


In [27]:
n = 20
matrix = get_symmetric(n, 10, seed=123)

for eps in np.logspace(-2, -7, 7):
    eigenvalues, iters = jacobi2(matrix, eps)
    diff = norm(np.array(eigenvalues) - np.array(sorted(np.linalg.eig(matrix)[0])))
    print( f"eps: {eps:.10e}   ", "iterations:", "{:6}    ".format(iters), "diff with np:", diff)

eps: 1.0000000000e-02    iterations:    513     diff with np: 5.577406553425442e-07
eps: 1.4677992676e-03    iterations:    542     diff with np: 3.917936084273757e-08
eps: 2.1544346900e-04    iterations:    580     diff with np: 6.108115412685773e-10
eps: 3.1622776602e-05    iterations:    617     diff with np: 7.301024914414115e-12
eps: 4.6415888336e-06    iterations:    643     diff with np: 5.789885455676922e-13
eps: 6.8129206906e-07    iterations:    664     diff with np: 2.2113424322591756e-13
eps: 1.0000000000e-07    iterations:    687     diff with np: 2.1705995879737016e-13
